### Binarne drzewo decyzyjne przy użyciu algorytmu ID3 Quinlana - Marceli Ptak
Tematem projektu jest stworzenie binarnego drzewa decyzyjnego z wykorzystaniem algorytmu ID3.<br /><br />
Problem jaki podejmuję dotyczy dylematu w kwestii zapisu na zajęcia boksu.<br />
Stworzone przeze mnie dane dotyczące potencjalnego uczestnika zajęć zawierają takie informacje jak:<br /><br />
1. Płeć:<br />
kobieta / mężczyzna,<br /><br />
2. Wiek:<br />
4 przedziały wiekowe: 12-18, 19-28, 29-40 oraz 40+,<br /><br />
3. Ile razy chcesz ćwiczyć w tygodniu?:<br />
częstotliwość w 3 przedziałach: 0-1, 2-3, 3+,<br /><br />
4. Kondycja:<br />
3 typy dotyczące aktywności sportowej: amator, medium, pro,<br /><br />
4. Podatność na kontuzje:<br />
określona w skali 3 stopniowej: duża, średnia oraz mała,<br /><br />
Konkluzją jest odpowiedź na pytanie: "Czy zapisać się na boks?",<br /> 
zawiera ona trzy możliwości: Tak, Nie, Raczej nie.<br /><br /> 
Dane to zbiór 100 ze wszystkich możliwych kombinacji (216) a konkluzja im przydzielana została w sposób arbitralny, choć intuicyjny.<br /><br />
Idealnym uczestnikiem zajęć boksu będzie więc młody wysportowany mężczyzna o małej podatności na kontuzje.<br /><br />
Osobą, która zaś na zajęciach nie powinna się pojawiać jest starsza kobieta nie uprawiająca na co dzień sportu o dużej podatności na kontuzje.

Importowanie koniecznych bibliotek:

In [1]:
import pandas as pd
import numpy as np

Wczytanie danych:<br /><br />
Wczytuję dane w formacie oddzielanym przecinkami (.csv) oraz wyświetlam pierwszych 5 rekordów:

In [2]:
dane = pd.read_csv("dane.csv", encoding= 'utf-8-sig')
print(dane.head())

      Plec   Wiek Ile razy chcesz cwiczyc w tygodniu Kondycja  \
0  Kobieta  12-18                                0-1   amator   
1  Kobieta  12-18                                0-1   amator   
2  Kobieta  12-18                                0-1   amator   
3  Kobieta  12-18                                0-1   medium   
4  Kobieta  12-18                                0-1   medium   

  Podatnosc na kontuzje Czy zapisac sie na boks?  
0                  duza                      Nie  
1               srednia                      Nie  
2                  mala               Raczej nie  
3                  duza                      Nie  
4               srednia               Raczej nie  


Zdefiniowanie funkcji "znajdź atrybut":<br /><br />
Funkcja pobiera jako argumenty dane, nazwę kolumny z konkluzją oraz wszyskie (3) jej możliwe wartości (możliwości).<br />
Zwraca najlepszą przesłankę spośród przekazanych danych (nazwa kolumny) oraz jej atrybut (wartość danej kolumny).<br /><br />
Aby odnaleźć najlepszy atrybut korzystam ze wzoru na entropię (tu funkcja posiłkuje się kolejną o nazwie entropia warunkowa) a następnie obliczam przyrost informacji jako różnicę pomiędzy entropią całkowitą a pojemnością informacyjną danego atrybutu. 

In [3]:
def znajdz_atrybut(dane, konkluzja, mozliwosci):
    przeslanki = dane.columns.drop(konkluzja)
    top_przeslanka = None
    top_atrybut = None
    top_przyrost_informacji = -1
    
    for przeslanka in przeslanki:           
        atrybuty = dane[przeslanka].unique()
        pojemnosc_informacji = 0

        for atrybut in atrybuty:
            dane_z_atrybutem = dane[dane[przeslanka] == atrybut]
            ilosc_rekordow_z_atrybutem = dane_z_atrybutem.shape[0]
            ilosc_rekordow = dane.shape[0]
            p = ilosc_rekordow_z_atrybutem/ilosc_rekordow
            entropia_p = entropia_warunkowa(dane_z_atrybutem, konkluzja, mozliwosci)
            pojemnosc_informacji = entropia_p * p
            ## Przyrost informacji, czyli o ile dana informacja "zmniejsza" entropię
            przyrost_informacji = entropia_calkowita(dane, konkluzja, mozliwosci) - pojemnosc_informacji
            if  top_przyrost_informacji < przyrost_informacji:
                top_przeslanka = przeslanka
                top_atrybut = atrybut
                top_przyrost_informacji = przyrost_informacji
    return top_przeslanka, top_atrybut

Zdefiniowanie funkcji "entropia całkowita":<br /><br />
Aby móc obliczyć przyrost informacji potrzebuję informacji o entropii całego zbioru.<br />
Jest to miara nieokreśloności (nieporządku) w zbiorze danych.<br />
Wzór z którego korzystam do obliczenia entropii to: I = SUMA(-plog2*p), gdzie p to prawdopodobieństwo wystapienia stanu.<br />
W przypadku binarnego drzewa decyzyjnego entropia określa ilość informacji potrzebnej do wyjaśnienia podziałów w drzewie.

In [4]:
def entropia_calkowita(dane, konkluzja, mozliwosci):
    entropia_calkowita = 0
    
    for i in mozliwosci:
        liczba_przykladow = dane[dane[konkluzja] == i].shape[0]
        n = dane.shape[0]
        x = liczba_przykladow/n
        entropia_i = entropia_i = -x * np.log2(x)
        entropia_calkowita = entropia_calkowita + entropia_i
        
    return entropia_calkowita

Zdefiniowanie funkcji "entropia warunkowa":<br /><br />
Analogicznie do entropii całkowitej obliczam entropię warunkową.<br />
Różnicą tych funkcji jest fakt, że entropia warunkowa pobiera dane z określonym atrybutem.

In [5]:
def entropia_warunkowa(dane_z_atrybutem, konkluzja, mozliwosci):
    entropia = 0
    for i in mozliwosci:
        liczba_przykladow = dane_z_atrybutem[dane_z_atrybutem[konkluzja] == i].shape[0]
        entropia_i = 0
        if liczba_przykladow != 0:
            ## Wzór na entropię: I = SUMA(-plog2*p)
            n = dane_z_atrybutem.shape[0]
            x = liczba_przykladow/n
            entropia_i = -x * np.log2(x)
    entropia = entropia + entropia_i
    return entropia

Zdefiniowanie funkcji "podziału":<br /><br />
Funkcja podziału pobiera dane a następnie zwraca najlepszy atrybut oraz podzielone dane.<br />
Atrybut jest zwracany wraz z przesłanką do której należy,<br />
a dane dzielone są na zawierające atrybut oraz nie. 

In [6]:
def podzial(dane):
    mozliwosci = dane["Czy zapisac sie na boks?"].unique()
    przeslanka, atrybut = znajdz_atrybut(dane, "Czy zapisac sie na boks?", mozliwosci)
    if (przeslanka is not None) and (atrybut is not None) :
        pytanie = przeslanka +", "+ atrybut
        
        dane1 = dane[dane[przeslanka] == atrybut]
        mozliwosci1 = dane1["Czy zapisac sie na boks?"].unique()
        if mozliwosci1.size < 1 :
            odpowiedz1 = "Czy zapisać się na boks?" + ", " + mozliwosci
            dane1 = odpowiedz1
       
        dane2 = dane[dane[przeslanka] != atrybut]
        mozliwosci2 = dane2["Czy zapisac sie na boks?"].unique()
        if mozliwosci2.size < 1 :
            odpowiedz2 = "Czy zapisać się na boks?" + ", " + mozliwosci
            dane2 = odpowiedz2
            
    return pytanie, dane1, dane2

In [7]:
pytanie1, dane1, dane2 = podzial(dane)
print(pytanie1)
print(dane1.head())
print(dane2.head())

Podatnosc na kontuzje, duza
       Plec   Wiek Ile razy chcesz cwiczyc w tygodniu Kondycja  \
0   Kobieta  12-18                                0-1   amator   
3   Kobieta  12-18                                0-1   medium   
6   Kobieta  12-18                                0-1      pro   
9   Kobieta  12-18                                2-3   amator   
12  Kobieta  12-18                                2-3   medium   

   Podatnosc na kontuzje Czy zapisac sie na boks?  
0                   duza                      Nie  
3                   duza                      Nie  
6                   duza                      Nie  
9                   duza                      Nie  
12                  duza                      Nie  
      Plec   Wiek Ile razy chcesz cwiczyc w tygodniu Kondycja  \
1  Kobieta  12-18                                0-1   amator   
2  Kobieta  12-18                                0-1   amator   
4  Kobieta  12-18                                0-1   medium   
5  

Pytanie, które przyniesie największe korzyści (najbardziej zmniejszy entropię) to to, czy potencjalny uczestnik ma dużą podatność na kontuzję.